In [1]:
import codecs
import re
import math
import collections
import numpy as np

In [2]:
def tokenization(text):
    
    doc = re.split(r' *[\;\:\—\<\>\\\"\.\,\!\?\&\=\+\(\)\{\}\[\]\r\n\t\« \»\“\”\„\/\d ]', text) 
    tokens = []
    for token in doc: 
        if token != '':
            tokens.append(token.lower())        
    return tokens

In [3]:
def tokenization_corpus(corpus):
    corpus_tokins = []
    for text in corpus:
        corpus_tokins.append(tokenization(text))
        #corpus_tokins.append([porter.stem(i.lower()) for i in tokenization(text) if i.lower() not in stop_words] )
    return corpus_tokins

In [4]:
text_ua = 'Кожна людина має право на освіту. Освіта повинна бути безплатною, хоча б початкова і загальна. Початкова освіта повинна бути обовязковою. Технічна і професійна освіта повинна бути загальнодоступною, а вища освіта повинна бути однаково доступною для всіх на основі здібностей кожного.Освіта повинна бути спрямована на повний розвиток людської особи і збільшення поваги до прав людини і основних свобод. Освіта повинна сприяти взаєморозумінню, терпимості і дружбі між усіма народами, расовими, або релігійними групами і повинна сприяти діяльності Організації Обєднаних Націй по підтриманню миру.Батьки мають право пріоритету у виборі виду освіти для своїх малолітніх дітей.'
text_ru = 'Такому жанру как статья присуща ширина практических обобщений, глубокий анализ фактов и явлений, четкая социальная направленность[источник не указан 3907 дней]. В статье автор рассматривает отдельные ситуации как часть более широкого явления. Автор аргументированно пишет о своей точке зрения.В статье выражается развернутая обстоятельная аргументированная концепция автора или редакции по поводу актуальной социологической проблемы. Также в статье журналист обязательно должен интерпретировать факты (это могут быть цифры, дополнительная информация, которая будет правильно расставлять акценты и ярко раскрывать суть вопроса).Отличительным аспектом статьи является её готовность. Если подготавливаемый материал так и не был опубликован (не вышел в тираж, не получил распространения), то такой труд относить к статье некорректно. Скорее всего данную работу можно назвать черновиком или заготовкой. Поэтому целью любой статьи является распространение содержащейся в ней информации.'

In [5]:
from collections import defaultdict

In [6]:
def getNgrams(tokens,n):
    count_ngram = defaultdict(int)
    for token in tokens:
        if n <= len(token):
            for i in range(len(token)-n+1):
                count_ngram[token[i:i+n]] +=1
    return count_ngram            
    

In [30]:
def getPr(tokens,n):
    sum_ru =0 
    sum_ua = 0
    sum_be = 0
    for token in tokens:
        if n <= len(token):
            for i in range(len(token)-n+1):
                sum_ru += tt_ru[token[i:i+n]]/len(tokens_ru)
                sum_ua += tt_ua[token[i:i+n]]/len(tokens_ua) 
                sum_be += tt_be[token[i:i+n]]/len(tokens_be) 
    return sum_ru,sum_ua,sum_be            
    

In [31]:
text_ua_test = 'Кожна людина має право вільно брати участь у культурному житті суспільства, втішатися мистецтвом, брати участь у науковому прогресі і користуватися його благами.'
tokens_ua_test = tokenization(text_ua_test)

In [208]:

def get_texts(path,n):
    texts = []
    #получаем каждый документ и создаем корпус
    for i in range(n):
        fileObj = codecs.open( path + str(i)+'.txt', "r", "utf_8_sig" )
        text = fileObj.read() 
        texts.append(text)
        fileObj.close()
    return texts

In [33]:
def all_corpus_tokens(corpus_tokens):
    all_tokens =[]
    for text in corpus_tokens:
        for word in text:
            all_tokens.append(word)
    return all_tokens

In [44]:
path_ru = './train/ru/'
path_ua = './train/ua/'
path_be = './train/be/'
texts_ru = get_texts(path_ru,8)
texts_ua = get_texts(path_ua,8)
texts_be = get_texts(path_be,8)

In [47]:
tokens_ru = tokenization_corpus(texts_ru)
tokens_ua = tokenization_corpus(texts_ua)
tokens_be = tokenization_corpus(texts_be)

In [48]:
tokens_ru = all_corpus_tokens(tokens_ru)
tokens_ua = all_corpus_tokens(tokens_ua)
tokens_be = all_corpus_tokens(tokens_be)

In [50]:
len(tokens_ru),len(tokens_ua),len(tokens_be)

(8070, 10548, 8146)

In [88]:
from sklearn.metrics import classification_report

In [86]:
def predict(answers):
    zeros = -1
    index = 0
    for i in range(len(answers)):
        if answers[i] > zeros:
            zeros = answers[i]
            index = i
    return index

In [298]:
k =3
tt_ua = getNgrams(tokens_ua,k)
tt_ru = getNgrams(tokens_ru,k)
tt_be = getNgrams(tokens_be,k)

In [299]:
len(tt_ru),len(tt_ua),len(tt_be)

(4069, 4032, 3649)

In [300]:
print(classification_report([1,2,0,1,2,1],[0,2,1,1,2,0],target_names=('ru','ua','be')))

              precision    recall  f1-score   support

          ru       0.00      0.00      0.00         1
          ua       0.50      0.33      0.40         3
          be       1.00      1.00      1.00         2

    accuracy                           0.50         6
   macro avg       0.50      0.44      0.47         6
weighted avg       0.58      0.50      0.53         6



In [301]:
path_be_test = './test/'
texts_be_test = get_texts(path_be_test,24)

In [302]:
tokens_be_test = tokenization_corpus(texts_be_test)

In [303]:
predicted = []
for text in tokens_be_test:
    predicted.append(predict(getPr(text,k)))

In [304]:
predicted

[2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [305]:
y_true = [2]*8+[1]*8+[0]*8

In [306]:
y_true

[2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [307]:
print(classification_report(y_true,predicted,target_names=('ru','ua','be')))

              precision    recall  f1-score   support

          ru       1.00      1.00      1.00         8
          ua       1.00      1.00      1.00         8
          be       1.00      1.00      1.00         8

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24

